In [ ]:
import time, random, json, os

import pandas as pd
from io import StringIO

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException, TimeoutException, StaleElementReferenceException

In [41]:
# 목록 리스트 생성
# TODO message 파일 어떻게 가져왔는지 문의 => 현재는 100개만 있음(합쳐서 더 많은 지역으로 확대)
with open("dong-number.txt", "r", encoding="utf-8") as f:
    data = f.read()

locations = json.loads(data)['locations']
keyword = '유모차+스토케'

search_list_urls = []

for lo in locations:
    dong_name = lo['name3']
    dong_id = lo['id']
    search_list_url = f'https://www.daangn.com/kr/buy-sell/?in={dong_name}-{dong_id}&search={keyword}'
    
    search_list_urls.append(search_list_url)

len(search_list_urls)

100

In [ ]:
# 해당 요소가 DOM에 없다고 나옴(StaleElementReferenceException) => 같은 코드를 반복해서 실행하면 처음은 되는데 두번째부터 되지 않음

# FROM GPT 원인
# 에러 원인은 “차단”이라기보다 DOM이 리렌더링되면서 이미 잡아둔 요소 핸들(참조)이 낡아졌기 때문이에요. 
# 당근 웹은 SPA/리액트 계열이라 스크롤·필터·광고 로딩 시 리스트가 통째로 갈아끼워지곤 합니다. 
# 그럼 직전에 잡은 WebElement가 StaleElementReferenceException을 던져요.
# 왜 처음엔 되고 다음 번엔 깨질까?
# 	•	페이지 로드 직후 1회는 리스트가 안정적이지만, 이후 가상 스크롤/무한스크롤·광고 삽입·반응형 레이아웃 때문에 섹션이 재구성됩니다.
# 	•	긴 CSS 체인(sprinkles_*, b4c4lz*)은 빌드마다 클래스명이 바뀌거나 DOM 트리가 달라지면 바로 무너집니다.

# item_urls = []
# driver = webdriver.Chrome()
# driver.get(search_list_urls[0])
# css_selector = '#main-content > div.wv2v230.sprinkles_paddingLeft_4_base__1byufe8g6.sprinkles_paddingLeft_10_small__1byufe8gr.sprinkles_paddingLeft_16_medium__1byufe8h4.sprinkles_paddingLeft_20_large__1byufe8h9.sprinkles_paddingRight_4_base__1byufe8e2.sprinkles_paddingRight_10_small__1byufe8en.sprinkles_paddingRight_16_medium__1byufe8f0.sprinkles_paddingRight_20_large__1byufe8f5.sprinkles_backgroundColor_layerDefault__1byufe8n.sprinkles_width_full_base__1byufe84q > div > section > div > div.b4c4lz3.sprinkles_width_full_base__1byufe84q.b4c4lz5 > a'
# item_a_links = driver.find_elements(By.CSS_SELECTOR, css_selector)
# print(item_a_links[0].get_attribute('href')) 

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=139.0.7258.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
0   chromedriver                        0x0000000104ac326c cxxbridge1$str$ptr + 2741972
1   chromedriver                        0x0000000104abb1dc cxxbridge1$str$ptr + 2709060
2   chromedriver                        0x00000001046054fc cxxbridge1$string$len + 90520
3   chromedriver                        0x000000010460b1b4 cxxbridge1$string$len + 114256
4   chromedriver                        0x000000010460d780 cxxbridge1$string$len + 123932
5   chromedriver                        0x000000010468ecfc cxxbridge1$string$len + 653720
6   chromedriver                        0x000000010468ddb8 cxxbridge1$string$len + 649812
7   chromedriver                        0x0000000104640d30 cx

In [ ]:
# 상품 상세 리스트 수집하여 json 파일로 제공 
# TODO 목록별 말고 그냥 상세 페이지만 리스트로 나오도록 변환

# ====== 동작 파라미터(필요시 조정) ======
OUT_PATH = "links_by_search_url.json"

# 스크롤/대기 관련
MAX_SCROLLS = 80            # 페이지당 스크롤 최대 횟수
STABLE_ROUNDS_TARGET = 3    # 새 링크가 안 늘어나는 횟수 연속되면 정지
SCROLL_PAUSE = (0.8, 1.6)   # 스크롤 후 대기(초)
URL_COOLDOWN = (5, 12)      # URL 간 쿨다운(초)

# 403 백오프(403 감지 시 대기 분)
BACKOFF_BASE_MIN = 5
BACKOFF_MAX_MIN  = 60

# ====== 드라이버 세팅 ======
opts = webdriver.ChromeOptions()
opts.add_argument("--window-size=1280,900")
opts.add_argument("--lang=ko-KR")
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=opts)

def polite_sleep(a, b):
    time.sleep(random.uniform(a, b))

def looks_like_403(html: str) -> bool:
    h = (html or "").lower()
    # 필요한 경우 사이트 특유 문구를 더 추가하세요.
    return ("403" in h and "forbidden" in h) or ("access denied" in h)

def backoff_sleep(cur_min):
    wait_min = min(cur_min, BACKOFF_MAX_MIN)
    print(f"[403 감지] {wait_min}분 대기 후 재시도")
    time.sleep(wait_min * 60)
    return min(wait_min * 2, BACKOFF_MAX_MIN)

def collect_search_article_links_on_page():
    """
    현재 페이지에서 `data-gtm="search_article"` <a>들의 href만 반환.
    맨 앞 1개는 제외.
    """
    anchors = driver.find_elements(By.CSS_SELECTOR, "a[data-gtm='search_article']")
    hrefs = []
    for a in anchors:
        try:
            href = a.get_attribute("href")
            if href:
                hrefs.append(href)
        except StaleElementReferenceException:
            # 리렌더링으로 인한 stale은 무시하고 다음 앵커 처리
            continue

    # 페이지마다 맨 앞 1개 제외
    #if hrefs:
    #    hrefs = hrefs[1:]
    
    return hrefs

def load_with_optional_backoff(url, backoff_min):
    """
    URL 로드 후 403이면 백오프-재시도 1번.
    반환값: (is_ok, next_backoff_min)
    """
    driver.get(url)
    if looks_like_403(driver.page_source):
        backoff_min = backoff_sleep(backoff_min)
        driver.get(url)
        if looks_like_403(driver.page_source):
            print(" - 403 지속: 이 URL은 건너뜁니다.")
            return False, backoff_min
    return True, backoff_min

def scroll_and_collect_all():
    """
    무한스크롤로 새 컨텐츠 로딩을 기다리며 링크 수집.
    새 링크가 더 이상 늘지 않으면 중단.
    """
    collected = set()
    stable_rounds = 0

    for i in range(MAX_SCROLLS):
        hrefs = collect_search_article_links_on_page()
        before = len(collected)

        for h in hrefs:
            collected.add(h)

        after = len(collected)
        if after == before:
            stable_rounds += 1
        else:
            stable_rounds = 0

        if stable_rounds >= STABLE_ROUNDS_TARGET:
            break

        # 스크롤 다운 & 대기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        polite_sleep(*SCROLL_PAUSE)

        # 스크롤 중간에 403 감지 시 현재 페이지 수집 중단
        if looks_like_403(driver.page_source):
            print(" - 스크롤 중 403 감지: 현재 페이지 수집을 중단합니다.")
            break

    return sorted(collected)

# ====== 메인 루프 ======
links_by_url = {}
# 이전 결과 존재 시 병합(중단/재개를 위해)
if os.path.exists(OUT_PATH):
    try:
        with open(OUT_PATH, "r", encoding="utf-8") as f:
            links_by_url = json.load(f)
            print(f"[복구] 기존 {len(links_by_url)}개 검색 URL 기록 로드")
    except Exception:
        pass

backoff_min = BACKOFF_BASE_MIN

for idx, url in enumerate(search_list_urls, start=1):
    print(f"\n[{idx}/{len(search_list_urls)}] 수집 시작: {url}")

    ok, backoff_min = load_with_optional_backoff(url, backoff_min)
    if not ok:
        continue

    # 스크롤 수집
    try:
        page_links = scroll_and_collect_all()
        print(f" - 수집 링크: {len(page_links)}개")
        links_by_url[url] = page_links

        # 중간 저장
        with open(OUT_PATH, "w", encoding="utf-8") as f:
            json.dump(links_by_url, f, ensure_ascii=False, indent=2)

    except WebDriverException as e:
        print(f" - 드라이버 오류로 중단: {e}")

    # URL 간 쿨다운
    polite_sleep(*URL_COOLDOWN)

driver.quit()

print(f"\n완료. 총 {len(links_by_url)}개 검색 URL에 대한 링크 수집 결과 저장 -> {OUT_PATH}")
# 확인용 샘플 출력
for u, lst in list(links_by_url.items())[:2]:
    print(f"\n{u}\n  count={len(lst)}")
    for x in lst[:5]:
        print("  -", x)


[1/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=정자동-1339&search=유모차+스토케
 - 수집 링크: 30개

[2/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=초월읍-1793&search=유모차+스토케
 - 수집 링크: 37개

[3/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=삼평동-1351&search=유모차+스토케
 - 수집 링크: 38개

[4/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=백현동-1352&search=유모차+스토케
 - 수집 링크: 38개

[5/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=석사동-1897&search=유모차+스토케
 - 수집 링크: 10개

[6/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=통진읍-1752&search=유모차+스토케
 - 수집 링크: 21개

[7/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=양촌읍-1754&search=유모차+스토케
 - 수집 링크: 29개

[8/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=구미동-1355&search=유모차+스토케
 - 수집 링크: 21개

[9/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=판교동-1350&search=유모차+스토케
 - 수집 링크: 25개

[10/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=서현동-4502&search=유모차+스토케
 - 수집 링크: 16개

[11/100] 수집 시작: https://www.daangn.com/kr/buy-sell/?in=금곡동-1353&sear

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=139.0.7258.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
0   chromedriver                        0x000000010485726c cxxbridge1$str$ptr + 2741972
1   chromedriver                        0x000000010484f1dc cxxbridge1$str$ptr + 2709060
2   chromedriver                        0x00000001043994fc cxxbridge1$string$len + 90520
3   chromedriver                        0x0000000104382f5c chromedriver + 208732
4   chromedriver                        0x00000001043a6018 cxxbridge1$string$len + 142516
5   chromedriver                        0x0000000104408cbc cxxbridge1$string$len + 547160
6   chromedriver                        0x00000001044218f8 cxxbridge1$string$len + 648596
7   chromedriver            